# Result Dispatch

Result Dispatch is a mechanism for distributing data to multiple workers. Let's understand these features with a practical example.

## Parallel Processing Example

Suppose we need to process multiple user queries in parallel. Each query needs to go through preprocessing, analysis, and then be aggregated. We can use Result Dispatch to efficiently handle this scenario.

### 1. Initialize

Let's start by importing the necessary packages.


In [2]:
from typing import Tuple
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.automa.args import Distribute, ResultDispatchRule

### 2. Input Dispatching

First, let's understand how to use `Distribute` when calling `arun()`. When you have multiple start workers and want to distribute different input values to each of them, you can wrap the input data in `Distribute()`.


In [ ]:
class ParallelProcessing(GraphAutoma):
    @worker(is_start=True)
    async def process_query_1(self, user_input: int) -> int:
        print(f"Processing query 1 with input: {user_input}")
        return user_input * 2  # 2
    
    @worker(is_start=True)
    async def process_query_2(self, user_input: int) -> int:
        print(f"Processing query 2 with input: {user_input}")
        return user_input * 3  # 6
    
    @worker(dependencies=["process_query_1", "process_query_2"], is_output=True)
    async def aggregate_results(self, result1: int, result2: int) -> int:
        print(f"Aggregating: {result1} + {result2}")
        return result1 + result2  # 8


Now let's run it with `Distribute` to distribute different values to the two start workers:

In [ ]:
automa = ParallelProcessing()
await automa.arun(user_input=Distribute([1, 2]))


Processing query 1 with input: 1
Processing query 2 with input: 2
Aggregating: 2 + 6


8

Great! As you can see, `Distribute([1, 2])` distributed the values `1` and `2` to `process_query_1` and `process_query_2` respectively, based on the order of the declaration of start workers.

> **Note**: The length of the list/tuple in `Distribute()` must match the number of start workers that accept the corresponding parameter. Otherwise, an error will be raised.

<div style="text-align: center; margin: 2rem 0;">
<hr style="border: none; border-top: 2px solid #e2e8f0;">
</div>

### 3. Result Dispatching

Now let's understand how to use `result_dispatch_rule=ResultDispatchRule.DISTRIBUTE` to distribute a worker's output to multiple downstream workers.

When a worker sets `result_dispatch_rule=ResultDispatchRule.DISTRIBUTE`, its return value must be an iterable (tuple or list). Each element in the return value will be distributed to the downstream workers that directly depend on this worker, in the order they are declared or added.


In [ ]:
class ResultDispatchingExample(GraphAutoma):
    @worker(is_start=True, result_dispatch_rule=ResultDispatchRule.AS_IS)
    async def generate_data(self, user_input: int) -> int:
        return user_input
    
    @worker(dependencies=["generate_data"], result_dispatch_rule=ResultDispatchRule.DISTRIBUTE)
    async def process_and_split(self, data: int) -> Tuple[int, int]:
        print(f"Processing data: {data}")
        # Return a tuple that will be distributed to downstream workers
        return data + 1, data + 2  # (2, 3)
    
    @worker(dependencies=["process_and_split"])
    async def worker_a(self, value: int) -> int:
        print(f"Worker A received: {value}")
        return value * 10  # 20
    
    @worker(dependencies=["process_and_split"])
    async def worker_b(self, value: int) -> int:
        print(f"Worker B received: {value}")
        return value * 20  # 60
    
    @worker(dependencies=["worker_a", "worker_b"], is_output=True)
    async def combine_results(self, result_a: int, result_b: int) -> int:
        print(f"Combining: {result_a} + {result_b}")
        return result_a + result_b  # 80


Wait, there's an issue with the above code. When `process_and_split` returns `(2, 3)`, we want to distribute these values to `worker_a` and `worker_b`. The `result_dispatch_rule` be set on the worker that **produces** the results to be distributed, not on the workers that **receive** them.

Now let's run it:

In [ ]:
automa = ResultDispatchingExample()
await automa.arun(user_input=1)

Processing data: 1
Worker A received: 2
Worker B received: 3
Combining: 20 + 60


80

Perfect! As you can see:

1. `process_and_split` returned `(2, 3)` and had `result_dispatch_rule=ResultDispatchRule.DISTRIBUTE`
2. The first value `2` was distributed to `worker_a` (the first downstream worker)
3. The second value `3` was distributed to `worker_b` (the second downstream worker)
4. Both workers processed their values and the results were combined

> **Important Notes**:
> - The worker that sets `result_dispatch_rule=ResultDispatchRule.DISTRIBUTE` must return an iterable (tuple or list)
> - The length of the return value must match the number of downstream workers that directly depend on this worker
> - The distribution order follows the order in which the downstream workers are declared in the graph

There are two mode of `ResultDispatchRule`:
- **AS_IS**: The result of the worker will be sent as a whole to each downstream worker that depends on it. At the same time, this is the default behavior.
- **Distribute**: The workers will distribute the current results in sequence to the corresponding workers one by one according to the order in which the downstream workers are declared or added.

## What have we leant?

Result Dispatch provides powerful mechanisms for parallel data processing:

- **`Distribute` in `arun()`**: Distributes input values to multiple start workers element-wise
- **`ResultDispatchRule`**: Allows a worker to distribute its output (must be iterable) to multiple downstream workers
    - **AS_IS**: Default mode to send the result of current worker as a whole
    - **Distribute**: Set to send the result of current worker according to the order in which the downstream workers are declared or added

These features enable efficient parallel processing and data flow management in complex workflows.